<a href="https://colab.research.google.com/github/RajeshworM/Yield_Modelling_Automation/blob/main/RF_Excess_Heat_Stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Upload your file
from google.colab import files
import pandas as pd
import numpy as np

uploaded = files.upload()   # Upload your existing dataset (CSV or Excel)

# Read uploaded file (auto-detect CSV/Excel)
filename = list(uploaded.keys())[0]
if filename.endswith(".csv"):
    df = pd.read_csv(filename)
elif filename.endswith((".xls", ".xlsx")):
    df = pd.read_excel(filename)
else:
    raise ValueError("Please upload a CSV or Excel file")

# -----------------------------
# STEP 2: Define reproductive weeks (34–38)
# -----------------------------
repro_weeks = range(34, 39)   # inclusive: 34, 35, 36, 37, 38

# -----------------------------
# STEP 3: PET & ER calculation
# -----------------------------
Ra = 15     # mm/day (simplified)
days_in_week = 7
Ra_weekly = Ra * days_in_week

def calc_reproductive_metrics(row, weeks):
    records = {}
    er_vals = []

    for w in weeks:
        try:
            rf = row[f"rf_week{w}"]
            tmax = row[f"tmax_week{w}"]
            tmin = row[f"tmin_week{w}"]
        except KeyError:
            continue

        # Mean temperature
        tmean = (tmax + tmin) / 2

        # PET (Hargreaves)
        pet = 0.0023 * (tmean + 17.8) * np.sqrt(max(tmax - tmin, 0.01)) * Ra_weekly

        # Excess Rainfall
        er = max(0, rf - pet)
        er_vals.append(er)

        # Save weekly ER
        records[f"ER_week{w}"] = er

    # Stage-level summary
    if er_vals:
        records.update({
            "ER_reproductive_sum": np.sum(er_vals),
            "ER_reproductive_max": np.max(er_vals),
            "ER_reproductive_mean": np.mean(er_vals),
            "ER_reproductive_heavy_count": np.sum(np.array(er_vals) > 50),  # weeks >50mm ER
            "Tmax_reproductive_mean": np.mean([row[f"tmax_week{w}"] for w in weeks if f"tmax_week{w}" in row]),
            "HighTemp_reproductive_count": np.sum([row[f"tmax_week{w}"] >= 35 for w in weeks if f"tmax_week{w}" in row])
        })
    return records

# -----------------------------
# STEP 4: Apply per district_id-year
# -----------------------------
all_metrics = []
for _, row in df.iterrows():
    metrics = {"district_id": row["district_id"], "year": row["year"]}
    metrics.update(calc_reproductive_metrics(row, repro_weeks))
    all_metrics.append(metrics)

stage_summary = pd.DataFrame(all_metrics)

# -----------------------------
# STEP 5: Merge with original data (optional)
# -----------------------------
final_df = pd.merge(df, stage_summary, on=["district_id", "year"], how="left")

# -----------------------------
# STEP 6: Save & download
# -----------------------------
output_file = "excess_rainfall_reproductive.csv"
final_df.to_csv(output_file, index=False)

files.download(output_file)


Saving data1.csv to data1.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>